## Clone Project

In [ ]:
# Clone the repo
!git clone https://<DRIVE>@github.com/java-master007/Adversarial-Representation-Learning-for-Medical-Imaging.git

Cloning into 'Adversarial-Representation-Learning-for-Medical-Imaging'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (490/490), done.
remote: Compressing objects: 100% (345/345), done.
remote: Total 490 (delta 281), reused 338 (delta 133), pack-reused 0
Receiving objects: 100% (490/490), 49.61 MiB | 38.20 MiB/s, done.
Resolving deltas: 100% (281/281), done.


In [ ]:
# Change to the correct directory
%cd Adversarial-Representation-Learning-for-Medical-Imaging/

/content/Adversarial-Representation-Learning-for-Medical-Imaging


In [ ]:
# Install requirements
# It will need restart on colab
! pip install -r requirements.txt

## Image Preparation

Requires that:
- malign.png be 3-channel
- normal.png be 3-channel
- malign_mask.png be one-channel

In [37]:
# Import required libraries
import cv2
import os
from skimage import io as img
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torchvision as tv
from PIL import Image, ImageDraw, ImageFilter

In [38]:
# Remove the 4 channel to collage image
def remove_4_channel(im_path, output_path):

    img = cv2.imread(im_path, cv2.IMREAD_UNCHANGED)

    # Transpose naive image to properly see it
    tranposed = img.transpose(2,0,1)

    # Transpose image again with only the 3 rgb channels to save
    output = tranposed[0:3].transpose(1,2,0)

    # Save new naive image (3-channels)
    cv2.imwrite(output_path, output)

In [39]:
# Resize image for hamronisation
def resize_image(im_path, percent_original, output_path):
    img = cv2.imread(im_path, cv2.IMREAD_UNCHANGED)
    
    print('Original Dimensions : ',img.shape)
    
    scale_percent = percent_original # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    print('Resized Dimensions : ',resized.shape)
    cv2.imwrite(output_path, resized)

In [40]:
# Make mask have 3 channels
def make_3_channels_mask(im_path, out_path):
  i = img.imread(im_path)
  new_i = []
  new_i.append(i)
  new_i.append(i)
  new_i.append(i)
  new_i = torch.tensor(np.array(new_i))
  tv.io.write_png(new_i, out_path)

In [41]:
# Crops the segmentation by its limits
def crop_segmentation(fp, outp):
  imag = cv2.imread(fp, cv2.IMREAD_UNCHANGED)
  imageObject = Image.open(fp)
  positions = np.nonzero(imag)

  top = positions[0].min()
  bottom = positions[0].max()
  left = positions[1].min()
  right = positions[1].max()

  cropped = imageObject.crop((left,top,right,bottom))
  cropped.save(outp)

In [43]:
# Makes a collage given the malign image, the malign mask, and the normal image
def make_collage(malign_pth, malign_mask_pth, normal_pth):

  # Reads malign base image
  malign = cv2.imread(malign_pth, cv2.IMREAD_UNCHANGED)

  # Convert mask to 3 channels
  make_3_channels_mask(malign_mask_pth, '/content/malign_mask3.png')
  malign_mask = cv2.imread('/content/malign_mask3.png', cv2.IMREAD_UNCHANGED)

  # Grab the image mask from the mass image
  masked = malign.copy()
  masked[malign_mask == 0] = 0
  cv2.imwrite('/content/segmented_mass.png', masked)

  # Crop both the mask, and the masked mass
  crop_segmentation('/content/segmented_mass.png', '/content/cropped_mass.png')
  crop_segmentation(malign_mask_pth, '/content/malign_mask_cropped.png')

  normal_image = Image.open(normal_pth)
  mass_to_paste = Image.open('/content/cropped_mass.png')
  mass_mask = Image.open('/content/malign_mask_cropped.png')

  # Creates collage and save
  back_im = normal_image.copy()
  #TODO: Calculate how to paste
  back_im.paste(mass_to_paste, (1000,1000), mass_mask)
  back_im.save('/content/collage.png', quality=95)

  # Creates collage mask
  collage_mask = Image.new("L", back_im.size, 0)
  collage_mask.paste(mass_mask, (1000,1000))
  collage_mask.save('/content/collage_mask.png', quality=95)

  # Deletes unecessary images
  try:
    os.remove('/content/malign_mask3.png')
    os.remove('/content/segmented_mass.png')
    os.remove('/content/cropped_mass.png')
    os.remove('/content/malign_mask_cropped.png')
  except OSError as e:
    print(f"FAILED\nFile: {e.filename}\nError: {e.strerror}")


In [44]:
make_collage(malign_pth='/content/malign.png', malign_mask_pth='/content/malign_mask.png', normal_pth='/content/normal.png')

## Harmonizer

In [ ]:
%cd MedSinGAN/

/content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN


In [ ]:
# Normal breast collage Harmonizer creation
!python main_train.py --train_mode harmonization --gpu 0 --train_stages 6 --lrelu_alpha 0.3 --niter 1000 --batch_norm --input_name /content/normal.png

Training model (TrainedModels/normal/2022_01_14_16_57_54_harmonization_niter_1000_lr_scale_0.1_nstages_6_BN_act_lrelu_0.3)
Training model with the following parameters:
	 number of stages: 6
	 number of concurrently trained stages: 3
	 learning rate scaling: 0.1
	 non-linearity: lrelu
Training on image pyramid: [torch.Size([1, 3, 31, 25]), torch.Size([1, 3, 47, 39]), torch.Size([1, 3, 71, 58]), torch.Size([1, 3, 108, 89]), torch.Size([1, 3, 165, 135]), torch.Size([1, 3, 250, 204])]

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:252: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,
stage [0/5]:: 100% 1000/1000 [00:29<00:00, 33.83it/s]
stage [1/5]::

In [ ]:
make_3_channels_mask('/content/collage_mask.png', '/content/collage_mask3.png')

In [ ]:
# Normal breast collage harmonisation
!python evaluate_model.py --gpu 0 --model_dir /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels/normal/2022_01_14_16_57_54_harmonization_niter_1000_lr_scale_0.1_nstages_6_BN_act_lrelu_0.3 --naive_img /content/collage.png

Loading models...
MASK SHAPE (250, 204, 3)
Done. Results saved at: /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels/normal/2022_01_14_16_57_54_harmonization_niter_1000_lr_scale_0.1_nstages_6_BN_act_lrelu_0.3/Evaluation_/content/collage.png


In [ ]:
a = img.imread('/content/collage.png')
a.shape

(2457, 1996, 3)

## Save Harmonizer Model

In [ ]:
# Import files to download zips
from google.colab import files

In [ ]:
# Zip the mlruns metrics to analyse
!zip -r /content/mlrun.zip /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/mlruns
files.download("/content/mlrun.zip")

  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/mlruns/ (stored 0%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/mlruns/.trash/ (stored 0%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/mlruns/0/ (stored 0%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/mlruns/0/683abe3f6e6048e5a9fa3c715cfdbddb/ (stored 0%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/mlruns/0/683abe3f6e6048e5a9fa3c715cfdbddb/artifacts/ (stored 0%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/mlruns/0/683abe3f6e6048e5a9fa3c715cfdbddb/metrics/ (stored 0%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/mlruns/0/683abe3f6e6048e5a9fa3c715cfdbddb/metrics/Discriminator Train Loss Fake (deflated 58%)
  adding: content/Adversarial-Representation-Learning-for-Medical

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Zip the best model analysed based on the mlruns
!zip -r /content/best_harmonisation_model.zip /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels
files.download("/content/best_harmonisation_model.zip")

  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels/ (stored 0%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels/normal/ (stored 0%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels/normal/2022_01_14_11_38_48_harmonization_niter_1000_lr_scale_0.1_nstages_6_BN_act_lrelu_0.3/ (stored 0%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels/normal/2022_01_14_11_38_48_harmonization_niter_1000_lr_scale_0.1_nstages_6_BN_act_lrelu_0.3/imresize.py (deflated 68%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels/normal/2022_01_14_11_38_48_harmonization_niter_1000_lr_scale_0.1_nstages_6_BN_act_lrelu_0.3/5/ (stored 0%)
  adding: content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels/normal/2022_01_14_11_38_48_harmonization_niter_1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Utils

In [ ]:
! rm -r /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/mlruns

In [ ]:
! rm -r /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN/TrainedModels